In [2]:
import json
import pathlib  
import os, sys
import pickle

import matplotlib.pyplot as plt
import cv2

import ollama_manager as om

import STAR_utils.visualization_tools.qa_visualization as qaviz
import main

import importlib


In [2]:
# please replace following data path to your local path
# all data can be download from our homepages (http://star.csail.mit.edu or https://bobbywu.com/STAR)

# root_dir = '/root/user/'
raw_video_dir = pathlib.Path('../data/datasets/action-genome/Charades_v1_480/')
raw_frame_dir = pathlib.Path('../data/datasets/action-genome/frames/')
annotation_dir = pathlib.Path('STAR_utils/annotations/STAR_classes/')
# pose_dir = root_dir + 'STAR/pose/'
# qa_train_dir = root_dir + 'STAR_train.json' 
# qa_val_dir = root_dir + 'STAR/STAR_val.json' 
# qa_test_dir = root_dir + 'STAR/STAR_test.json' 
save_video_dir = pathlib.Path('/home/lusha/visualization_tmp')
fps = pickle.load(open(annotation_dir / 'video_fps','rb'))


In [ ]:
import batch_processor

importlib.reload(main)
importlib.reload(batch_processor)

SEED = 39
url = os.environ.get("OLLAMA_URL", "http://lusha_ollama:11435")

sys_file_path = '../data/prompts/img_captioning/system_prompt.txt'
sys_prompt = main._load_prompt_fromfile(sys_file_path)
ollama_params={
            "model": "llama3.2",
            "system": sys_prompt,
            "stream": True,
            "options": {
                "num_ctx": 10240,
                "temperature": 0.1,
                "num_predict": 512,
                "seed": SEED,
            },
        }

client = om.OllamaRequestManager(url, ollama_params)



In [ ]:
output_file_path = '../outputs/gen_frames.jsonl'
main.streaming_frame_generation(client, output_file_path)


In [3]:
with open('../outputs/responses_genframes:7b_202504010_22:47:00.jsonl', 'r') as f:
    content = [json.loads(l) for l in f.readlines()]
    

content[0]['qid']


'Interaction_T1_13'

In [4]:
content[0].keys()


dict_keys(['qid', 'chat_history', 'stsg'])

In [6]:
len(content)


16

In [79]:
content[0]['chat_history'][0].keys()


dict_keys(['role', 'content', 'images'])

In [82]:
content[0]['chat_history'][1].keys()


dict_keys(['content', 'role'])

In [81]:
content[0]['chat_history'][2].keys()


dict_keys(['role', 'content'])

In [80]:
content[0]['chat_history'][3].keys()


dict_keys(['role', 'content'])

In [96]:
content[0]['stsg']


'\nFrame 000198:\n\nyoung_man ---- standing_in_front_of ---- sofa\nsofa ---- sitting_on ---- cushions\ncushions ---- on ---- sofa\nsofa ---- positioned_infront_of ---- wall\nwall ---- plain_color ---- white\nwall ---- mounted_with ---- electrical_outlet\nelectrical_outlet ---- attached_to ---- wall\nwall ---- mounted_with ---- light_switch\nlight_switch ---- attached_to ---- wall\nwall ---- background_for ---- sofa\nwall ---- positioned_behind ---- young_man\nyoung_man ---- looking_at ---- sofa\nsofa ---- positioned_on ---- floor\nfloor ---- covered_with ---- tiles\ntiles ---- arranged_in_squares ---- floor\n\nFrame 000212:\n\nman ---- holding ---- backpack\nman ---- standing_in_front_of ---- sofa\nsofa ---- positioned_on ---- floor\nsofa ---- containing ---- cushions\ncushion ---- on ---- sofa\ncushion ---- part_of ---- sofa\nbackpack ---- held_by ---- man\nsofa ---- adjacent_to ---- wall\nwall ---- behind ---- man\nelectrical_outlet ---- attached_to ---- wall\nlight_fixture ---- atta

In [72]:
content[0]['chat_history'][0]


{'role': 'user',
 'content': '    Look carefully at this image and identify all objects and relationships present.\n\n    First, list all distinct objects you can detect in the image. Be thorough and specific with your object labels (e.g., "young woman" rather than just "person", "wooden chair" rather than just "chair").\n\n    Then, describe the key relationships between these objects in free-form text. Consider:\n    - Spatial relationships (above, below, behind, inside, etc.)\n    - Action-based relationships (holding, looking at, sitting on, etc.)\n    - Physical connections (attached to, part of, touching, etc.)\n    - Relative positions (next to, between, surrounding, etc.)\n\n    Think step by step.'}

In [ ]:
frames = list(frames for frames in main.generate_frames(iters=2))
frames


In [ ]:
fps


In [5]:

with open("../data/datasets/STAR/STAR_annotations/STAR_val.json") as in_file:
    star_data = json.load(in_file)


In [ ]:
test_id = 222
test_sample = star_data[test_id]
list(test_sample.keys())


In [ ]:
test_sample['answer']


In [ ]:
test_sample['situations']


In [ ]:
graph_dict = dict()

for frame_id, frame_info in test_sample['situations'].items():
    graph_dict[frame_id] = []
    for i in range(len(frame_info['rel_pairs'])):

        assert len(frame_info['rel_pairs']) == len(frame_info['rel_labels'])
        
        entry = (frame_info['rel_pairs'][i][0], frame_info['rel_labels'][i], frame_info['rel_pairs'][i][1])
        graph_dict[frame_id].append(entry)

graph_dict


In [ ]:
qaviz.Vis_Question_Answer_Options(test_sample)


In [ ]:
test_sample.keys()


In [ ]:
test_sample['video_id']


In [ ]:
test_sample['start'], test_sample['end']


In [ ]:
test_sample['situations']


In [14]:
raw_video_dir = pathlib.Path('../data/datasets/action-genome/Charades_v1_480/')
raw_frame_dir = pathlib.Path('../data/datasets/action-genome/frames/')
save_video_dir = pathlib.Path('/home/lusha/')


In [ ]:
frame = cv2.imread(raw_frame_dir / f"{test_sample['video_id']}.mp4" / f"000228.png")
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Display using Matplotlib
plt.imshow(frame)
plt.axis('off')  # Hide axes
plt.show()


In [ ]:
qaviz.Vis_SituationGraph(test_sample, 1_000)


In [ ]:
qaviz.Vis_Video(test_sample, raw_video_dir, save_video_dir)


In [ ]:
from STAR_utils.visualization_tools import vis_utils

frame_ids = vis_utils.sample_frames(list(test_sample['situations'].keys()), 10)
frame_ids


In [ ]:
import importlib

importlib.reload(vis_utils)


In [ ]:
def load_frames(frame_list, frame_dir):
    select = []
    for i in range(len(frame_list)):
        frame = cv2.imread(frame_dir / f"{frame_list[i]}.png")
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        select.append(frame)
        
    return select
    

frame_dir = raw_frame_dir / f"{test_sample['video_id']}.mp4"
frames = load_frames(frame_ids, frame_dir)

print(len(frames))
frames[0]


In [ ]:
qaviz.Vis_Keyframes(test_sample,fps,200,raw_frame_dir)


In [ ]:

qaviz.Vis_Keyframes(test_sample,fps,10,raw_frame_dir)


In [ ]:
qaviz.Vis_Keyframes(test_sample,fps,10,raw_frame_dir)


In [ ]:
qaviz.Vis_Keyframes(test_sample,fps,4,raw_frame_dir)


In [ ]:
qaviz.Vis_Keyframes(test_sample,fps, 15, raw_frame_dir)


In [ ]:
qaviz.Vis_Box(test_sample,fps, 5, raw_frame_dir)


In [ ]:
import base64

base64_encodings = []
for f_id in frame_ids:
    frame_path = raw_frame_dir / f"{test_sample['video_id']}.mp4" / f"{f_id}.png"
    with open(frame_path, "rb") as f:
        img_bytes = f.read()
        base64_encodings.append(base64.b64encode(img_bytes).decode('utf-8'))
        
len(base64_encodings)


In [ ]:
import ollama_manager as om


url = os.environ.get("OLLAMA_URL", "http://lusha_ollama:11435")

ollama_params = {
    "model": "gemma3:4b"
}

client = om.OllamaRequestManager(url, ollama_params)
r = client.ollama_completion_request(
    endpoint='generate',
    payload={
        "model": "gemma3:4b",
        "prompt": "how are you"
    }
)


In [ ]:
prompt1 = """\
Look carefully at this image and identify all objects and relationships present.

First, list all distinct objects you can detect in the image. Be thorough and specific with your object labels (e.g., "young woman" rather than just "person", "wooden chair" rather than just "chair").

Then, describe the key relationships between these objects in free-form text. Consider:
- Spatial relationships (above, below, behind, inside, etc.)
- Action-based relationships (holding, looking at, sitting on, etc.)
- Physical connections (attached to, part of, touching, etc.)
- Relative positions (next to, between, surrounding, etc.)

Think step by step.
"""

msg1 = {
    "role": "user",
    "content": prompt1,
}
resp = client.ollama_completion_request(
            endpoint='chat',
            payload={
                "model": "gemma3:4b",
                "messages": [msg1],
                "options": {
                    "num_ctx": 10_000
                }
            }
        )

prompt2 = \
    """\
    Thank you. Now organize the objects and relationships you identified into a formal scene graph using this format:
    object1 ---- relationship ---- object2
    
    The list of relationship predicates should be introduced by the tag <scene_graph> and terminated by the tag </scene_graph>
    For example:
    woman ---- sitting_on ---- chair
    dog ---- lying_under ---- table
    book ---- on_top_of ---- shelf

    Please follow these guidelines:
    1. Create at least 10 relationship triplets (more if the image is complex)
    2. Use specific and consistent object labels
    3. Use concise but descriptive relationship terms (connect words with underscores)
    4. Include all meaningful relationships between objects
    5. Verify that all objects you identified in step 1 appear in at least one relationship

    Your scene graph:\
    """

messages = [msg1, resp, {"role": "user", "content": prompt2}]
resp = client.ollama_completion_request(
            endpoint='chat',
            payload={
                "model": "gemma3:4b",
                "messages": messages,
                "options": {
                    "num_ctx": 10_000
                }
            }
        )

resp


In [ ]:
resp


In [5]:
import re


In [ ]:
test_str = "Okay, here’s the scene graph representation of the image, following your specified format and guidelines:\n\n<scene_graph>\nman ---- standing_in_front_of ---- sofa\nsofa ---- positioned_in_front_of ---- coffee_table\ncoffee_table ---- covered_by ---- tablecloth\ntablecloth ---- on ---- coffee_table\nman ---- looking_at ---- coffee_table\nlamp ---- on ---- metal_stand\nmetal_stand ---- supporting ---- lamp\nlamp ---- positioned_on ---- coffee_table\nwindow ---- containing ---- curtain\ncurtain ---- hanging_in ---- window\nwindow ---- mounted_on ---- wall\nwall ---- supporting ---- window\nman ---- facing ---- coffee_table\nsofa ---- adjacent_to ---- coffee_table\n</scene_graph>\n\nI’ve aimed for clarity and accuracy in describing the relationships. Let me know if you’d like me to refine any of the triplets or add more detail!"

# the ?s: specify the re.DOTALL flag
pattern = "(?<=<scene_graph>)(?s:.+)(?=</scene_graph)"
res = re.search(pattern,test_str)
if res:
    print(res.group(0))
    
res
